**This script contains the figure generating functions for Fig 2.B**

In [ ]:
import os
from pathlib import Path
import json

# 1. Find the Repo Root dynamically
# Walks up folders until it finds the README.md file
_root = next(p for p in Path.cwd().parents if (p / "README.md").exists())
REPO_ROOT = str(_root)

# 2. Add to sys.path so standard 'import' statements work
import sys
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

# Load the Data Root from the JSON file
with open(Path(REPO_ROOT) / "data_config.json", "r") as f:
    config = json.load(f)
    DATA_ROOT = config["DATA_ROOT"]

print(f"Data is being pulled from: {DATA_ROOT}")
print(f"Repo root identified as: {REPO_ROOT}")

In [ ]:
############### LOADING DATA ###############

%run "$REPO_ROOT/config/predictions/model_load.py"

In [ ]:
############### ALTERNATIVE METHOD (MEAN-EXPRESSION STRATIFIED) ###############

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import pearsonr

MODEL_COLORS = {
    'MLR': '#2ca02c', 
    'XGBRFRegressor': '#ff7f0e', 
    'RNN': '#1f77b4'
}


def figure_2_stratified_with_outliers_mean(y_true, predictions_dict, 
                                     output_path=f"{DATA_ROOT}/Saved figures/Production_model_figures/Fig2/figure_2_diagnostics.png"):
    """
    Diagnostic suite: Stratifies by expression, quantifies bias, and identifies 
    top 5 problematic genes.
    """
    set_publication_style()
    
    # 1. Setup Data & Strata
    gene_means = y_true.mean(axis=0)
    gene_names = y_true.columns
        
    # Define Expression Strata
    low_thresh, high_thresh = 0.5, 3.0
    strata_masks = {
        'Low Mean Expression (<0.5)': gene_means <= low_thresh,
        'Medium Mean Expression': (gene_means > low_thresh) & (gene_means <= high_thresh),
        'High Mean Expression (>3.0)': gene_means > high_thresh
    }
    
    # Dynamically get model names from dict (fixes KeyError)
    model_names = list(predictions_dict.keys())
    
    # 3x3 Grid: Rows = Strata, Cols = Models
    # Using a larger vertical size (18) to accommodate 3 rows
    fig, axes = plt.subplots(len(strata_masks), len(model_names), 
                             figsize=(18, 15), sharex='col', sharey='row')
    
    for row_idx, (stratum_name, mask) in enumerate(strata_masks.items()):
        stratum_genes = gene_names[mask]
        # Use .values for faster math and to avoid indexing errors
        y_true_stratum = y_true.loc[:, mask].values 
        
        for col_idx, model_name in enumerate(model_names):
            ax = axes[row_idx, col_idx]
            y_pred_raw = predictions_dict[model_name]
            
            # Safe indexing for DF or Array
            y_pred_stratum = y_pred_raw.loc[:, mask].values if hasattr(y_pred_raw, 'loc') else y_pred_raw[:, mask]
            
            # --- MATH ---
            residuals = (y_true_stratum - y_pred_stratum)
            gene_bias = residuals.mean(axis=0) # Mean error per gene
            
            res_flat = residuals.ravel()
            pred_flat = y_pred_stratum.ravel()
            
            # Identify Top 5 Biased Genes
            abs_bias = np.abs(gene_bias)
            top_5_idx = np.argsort(abs_bias)[-5:][::-1]
            top_5_genes = stratum_genes[top_5_idx]
            top_5_vals = gene_bias[top_5_idx]

            # plotting
            # Subsampling 50k points for performance while maintaining visual density
            if len(res_flat) > 50000:
                idx = np.random.choice(len(res_flat), 50000, replace=False)
                ax.scatter(pred_flat[idx], res_flat[idx], alpha=0.5, s=5, 
                           color=MODEL_COLORS.get(model_name, '#1f77b4'), edgecolors='none')
            else:
                ax.scatter(pred_flat, res_flat, alpha=0.5, s=5, 
                           color=MODEL_COLORS.get(model_name, '#1f77b4'), edgecolors='none')

            # Zero-baseline
            ax.axhline(0, color='k', linestyle='--', lw=1, alpha=0.5)
            
            # --- WHEAT BOX ---
            outlier_str = "\n".join([f"{g}: {v:.2f}" for g, v in zip(top_5_genes, top_5_vals)])
            
            # Removing bold markers and using standard weight
            textstr = (f"Global Bias = {np.mean(res_flat):.4f}\n"
                       f"Residual SD = {np.std(res_flat):.4f}\n"
                       f"----------\n"
                       f"Top 5 Biased Genes:\n{outlier_str}")
            
            # bbox 'edgecolor' set to 'none' removes the black border
            ax.text(0.95, 0.95, textstr, transform=ax.transAxes, 
                    fontsize=10, verticalalignment='top', horizontalalignment = 'right', fontfamily='monospace',
                    fontweight='normal',
                    bbox=dict(boxstyle='round,pad=0.5', facecolor='wheat', alpha=0.8, edgecolor='none'))

            # --- LABELS & TITLES ---
            if row_idx == 0:
                ax.set_title(model_name, fontsize=13, fontweight='bold')
            
            if col_idx == 0:
                ax.set_ylabel(f"{stratum_name}\nResiduals (Log10)", fontsize=12, fontweight='bold')
            else:
                ax.set_ylabel('') 
                
            if row_idx == 2:
                ax.set_xlabel('Predicted Expression (Log10)', fontsize=12, fontweight='bold')
            
            ax.grid(True, alpha=0.3)
            ax.set_ylim(-4, 4) # range for honest cross-model comparison
            ax.set_xlim(0, 6)  # matches fig 1 X/Y limits

    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"Figure 2 saved to {output_path}")
    plt.show()

In [ ]:
############### ALTERNATIVE METHOD (CV-STRATIFIED) WITH RMSE/MAE ###############

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error

MODEL_COLORS = {
    'MLR': '#2ca02c', 
    'XGBRFRegressor': '#ff7f0e', 
    'RNN': '#1f77b4'
}

def set_publication_style():
    """Apply consistent publication-quality styling."""
    import seaborn as sns
    sns.set_style("whitegrid")
    plt.rcParams.update({
        'figure.dpi': 300,
        'savefig.dpi': 300,
        'font.size': 11,
        'axes.labelsize': 12,
        'axes.titlesize': 13,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'legend.fontsize': 10,
        'axes.linewidth': 1.2,
        'xtick.major.width': 1.2,
        'ytick.major.width': 1.2,
        'xtick.major.size': 5,
        'ytick.major.size': 5,
    })

def figure_2_stratified_cv(y_true, predictions_dict, 
                          output_path=f"{DATA_ROOT}/Saved figures/Production_model_figures/Fig2/figure_2_diagnostics_with_RMSE_MAE.png"):
    """
    Diagnostic suite: Stratifies by Coefficient of Variation (CV).
    Shows RMSE and MAE metrics for each stratum and model.
    
    Parameters
    ----------
    y_true : pd.DataFrame
        True gene expression values (samples × genes)
    predictions_dict : dict
        Dictionary mapping model names to prediction arrays/DataFrames
    output_path : str
        Path to save the output figure
        
    Returns
    -------
    fig, axes : matplotlib figure and axes objects
    """
    set_publication_style()
    
    # 1. Setup Data & Strata using CV (Variance / Mean)
    gene_means = y_true.mean(axis=0)
    gene_stds = y_true.std(axis=0)
    # Calculate CV; handle potential zeros by replacing with NaN then ignoring
    gene_cv = (gene_stds / gene_means).replace([np.inf, -np.inf], np.nan)
    gene_names = y_true.columns
    
    # Determine Quantiles for balanced n (33rd and 67th percentiles)
    q1, q2 = np.nanpercentile(gene_cv, [33.33, 66.67])
        
    # Define Variance Strata based on CV percentiles
    strata_masks = {
        f'Low Variance (CV < {q1:.2f})': gene_cv <= q1,
        f'Medium Variance ({q1:.2f}-{q2:.2f})': (gene_cv > q1) & (gene_cv <= q2),
        f'High Variance (CV > {q2:.2f})': gene_cv > q2
    }
    
    # Dynamically get model names from dict
    model_names = list(predictions_dict.keys())
    
    # 3x3 Grid: Rows = Strata, Cols = Models
    fig, axes = plt.subplots(len(strata_masks), len(model_names), 
                             figsize=(18, 15), sharex='col', sharey='row')
    
    for row_idx, (stratum_name, mask) in enumerate(strata_masks.items()):
        # Ensure we drop NaNs if CV couldn't be calculated for specific genes
        mask = mask.fillna(False)
        stratum_genes = gene_names[mask]
        y_true_stratum = y_true.loc[:, mask].values 
        
        for col_idx, model_name in enumerate(model_names):
            ax = axes[row_idx, col_idx]
            y_pred_raw = predictions_dict[model_name]
            y_pred_stratum = y_pred_raw.loc[:, mask].values if hasattr(y_pred_raw, 'loc') else y_pred_raw[:, mask]
            
            # --- COMPUTE METRICS ---
            residuals = (y_true_stratum - y_pred_stratum)
            
            # Flatten for overall stratum metrics
            y_true_flat = y_true_stratum.ravel()
            y_pred_flat = y_pred_stratum.ravel()
            res_flat = residuals.ravel()
            
            # Calculate RMSE and MAE
            rmse = np.sqrt(mean_squared_error(y_true_flat, y_pred_flat))
            mae = mean_absolute_error(y_true_flat, y_pred_flat)
            
            # Additional useful metrics
            pearson_r, _ = pearsonr(y_true_flat, y_pred_flat)
            residual_std = np.std(res_flat)
            mean_bias = np.mean(res_flat)
            
            # Plotting - subsample if too many points for visualization
            if len(res_flat) > 50000:
                idx = np.random.choice(len(res_flat), 50000, replace=False)
                ax.scatter(y_pred_flat[idx], res_flat[idx], alpha=0.5, s=5, 
                           color=MODEL_COLORS.get(model_name, '#1f77b4'), edgecolors='none')
            else:
                ax.scatter(y_pred_flat, res_flat, alpha=0.5, s=5, 
                           color=MODEL_COLORS.get(model_name, '#1f77b4'), edgecolors='none')

            # Zero-baseline
            ax.axhline(0, color='k', linestyle='--', lw=1, alpha=0.5)
            
            # --- METRICS BOX ---
            textstr = (f"RMSE = {rmse:.4f}\n"
                      f"MAE = {mae:.4f}\n"
                      f"----------\n"
                      f"Pearson's R = {pearson_r:.4f}\n"
                      f"Mean Bias = {mean_bias:.4f}\n"
                      f"Residual SD = {residual_std:.4f}\n"
                      f"----------\n"
                      f"N genes = {len(stratum_genes):,}")
            
            ax.text(0.95, 0.95, textstr, transform=ax.transAxes, 
                    fontsize=10, verticalalignment='top', horizontalalignment='right', 
                    fontfamily='monospace', fontweight='normal',
                    bbox=dict(boxstyle='round,pad=0.5', facecolor='wheat', alpha=0.8, edgecolor='none'))

            # --- LABELS & TITLES ---
            if row_idx == 0:
                ax.set_title(model_name, fontsize=13, fontweight='bold')
            
            if col_idx == 0:
                ax.set_ylabel(f"{stratum_name}\nResiduals (Log10)", fontsize=12, fontweight='bold')
            else:
                ax.set_ylabel('') 
                
            if row_idx == 2:
                ax.set_xlabel('Predicted Expression (Log10)', fontsize=12, fontweight='bold')
            
            ax.grid(True, alpha=0.3)
            ax.set_ylim(-4, 4) 
            ax.set_xlim(0, 6)  

    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"Figure 2 (CV-Stratified with RMSE/MAE) saved to {output_path}")
    plt.show()
    
    return fig, axes

In [ ]:
############### PLOTTING STEP (CV-STRATIFIED) ###############

figure_2_stratified_cv(y_test, predictions_test, 'figure_2_diagnostics_with_CR.png')

In [ ]:
############### PLOTTING STEP (MEAN EXPRESSION -STRATIFIED) ###############

figure_2_stratified_with_outliers_mean(y_test, predictions, 'figure_2_stratified_with_outliers_mean.png')